<a href="https://colab.research.google.com/github/pksX01/sarvam-llm-finetuning/blob/main/sarvam_1_llm_finetuning_bhojpuri_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q bitsandbytes transformers peft accelerate datasets trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from google.colab import userdata
import os
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
os.environ["WANDB_API_KEY"] = userdata.get("WANDB")

In [ ]:
import wandb
wandb.init(project="sarvam_1_llm_finetuning_bhojpuri_qa")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: pksx01. Use `wandb login --relogin` to force relogin


In [ ]:
from datasets import load_dataset

ds = load_dataset("SatyamDev/alpaca_data_cleaned_bhojpuri")

README.md:   0%|          | 0.00/5.31k [00:00<?, ?B/s]

alpaca_data_cleaned_bhojpuri.csv:   0%|          | 0.00/95.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [ ]:
type(ds)

datasets.dataset_dict.DatasetDict

In [ ]:
ds['train'][0]

{'instruction': 'स्वस्थ रहे खातिर तीन गो टिप्स दीं।',
 'input': None,
 'output': '1. संतुलित अवुरी पौष्टिक आहार खाईं: सुनिश्चित करीं कि आपके भोजन में कई प्रकार के फल अवुरी सब्जी, दुबला प्रोटीन, साबुत अनाज अवुरी स्वस्थ वसा शामिल होखे। एहसे आपके शरीर के सबसे निमन तरीका से काम करे खाती जरूरी पोषक तत्व मिले में मदद मिलेला अवुरी पुरान बेमारी से बचाव में मदद मिल सकता।\n\n2. नियमित शारीरिक गतिविधि में शामिल होखे के चाही: हड्डी, मांसपेशी अवुरी हृदय संबंधी स्वास्थ्य के मजबूत बनावे खाती व्यायाम बहुत जरूरी बा। हर हफ्ता कम से कम 150 मिनट मध्यम एरोबिक व्यायाम भा 75 मिनट के जोरदार व्यायाम के लक्ष्य राखीं.\n\n3. पर्याप्त नींद लेवे के बा : शारीरिक अवुरी मानसिक भलाई खाती पर्याप्त गुणवत्ता वाला नींद बहुत जरूरी बा। इ मूड के नियंत्रित करे में मदद करेला, संज्ञानात्मक कामकाज में सुधार करेला अवुरी स्वस्थ विकास अवुरी प्रतिरक्षा के कामकाज के समर्थन करेला। हर रात 7-9 घंटा नींद के लक्ष्य राखीं।'}

In [ ]:
type(ds['train'])

datasets.arrow_dataset.Dataset

In [ ]:
len(ds['train'])

51760

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-20751e10-44d1-7459-c794-e4ee697994e6)


In [ ]:
def merge_columns(row):
    data = list()
    data.append({"content": row["instruction"], "role": "user"})
    data.append({"content": row["output"], "role": "assistant"})
    row["messages"] = data
    #row['instr_op'] = row['instruction'] + " -> " + str(row['output'])
    return row

In [ ]:
ds['train'] = ds['train'].map(merge_columns)

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [ ]:
ds['train'][0]

{'instruction': 'स्वस्थ रहे खातिर तीन गो टिप्स दीं।',
 'input': None,
 'output': '1. संतुलित अवुरी पौष्टिक आहार खाईं: सुनिश्चित करीं कि आपके भोजन में कई प्रकार के फल अवुरी सब्जी, दुबला प्रोटीन, साबुत अनाज अवुरी स्वस्थ वसा शामिल होखे। एहसे आपके शरीर के सबसे निमन तरीका से काम करे खाती जरूरी पोषक तत्व मिले में मदद मिलेला अवुरी पुरान बेमारी से बचाव में मदद मिल सकता।\n\n2. नियमित शारीरिक गतिविधि में शामिल होखे के चाही: हड्डी, मांसपेशी अवुरी हृदय संबंधी स्वास्थ्य के मजबूत बनावे खाती व्यायाम बहुत जरूरी बा। हर हफ्ता कम से कम 150 मिनट मध्यम एरोबिक व्यायाम भा 75 मिनट के जोरदार व्यायाम के लक्ष्य राखीं.\n\n3. पर्याप्त नींद लेवे के बा : शारीरिक अवुरी मानसिक भलाई खाती पर्याप्त गुणवत्ता वाला नींद बहुत जरूरी बा। इ मूड के नियंत्रित करे में मदद करेला, संज्ञानात्मक कामकाज में सुधार करेला अवुरी स्वस्थ विकास अवुरी प्रतिरक्षा के कामकाज के समर्थन करेला। हर रात 7-9 घंटा नींद के लक्ष्य राखीं।',
 'messages': [{'content': 'स्वस्थ रहे खातिर तीन गो टिप्स दीं।',
   'role': 'user'},
  {'content': '1. संतुलित अवुरी प

In [ ]:
ds['train']['messages'][:3]

[[{'content': 'स्वस्थ रहे खातिर तीन गो टिप्स दीं।', 'role': 'user'},
  {'content': '1. संतुलित अवुरी पौष्टिक आहार खाईं: सुनिश्चित करीं कि आपके भोजन में कई प्रकार के फल अवुरी सब्जी, दुबला प्रोटीन, साबुत अनाज अवुरी स्वस्थ वसा शामिल होखे। एहसे आपके शरीर के सबसे निमन तरीका से काम करे खाती जरूरी पोषक तत्व मिले में मदद मिलेला अवुरी पुरान बेमारी से बचाव में मदद मिल सकता।\n\n2. नियमित शारीरिक गतिविधि में शामिल होखे के चाही: हड्डी, मांसपेशी अवुरी हृदय संबंधी स्वास्थ्य के मजबूत बनावे खाती व्यायाम बहुत जरूरी बा। हर हफ्ता कम से कम 150 मिनट मध्यम एरोबिक व्यायाम भा 75 मिनट के जोरदार व्यायाम के लक्ष्य राखीं.\n\n3. पर्याप्त नींद लेवे के बा : शारीरिक अवुरी मानसिक भलाई खाती पर्याप्त गुणवत्ता वाला नींद बहुत जरूरी बा। इ मूड के नियंत्रित करे में मदद करेला, संज्ञानात्मक कामकाज में सुधार करेला अवुरी स्वस्थ विकास अवुरी प्रतिरक्षा के कामकाज के समर्थन करेला। हर रात 7-9 घंटा नींद के लक्ष्य राखीं।',
   'role': 'assistant'}],
 [{'content': 'तीनों प्राथमिक रंग का हवें?', 'role': 'user'},
  {'content': 'तीनों प्राथम

In [ ]:
ds['train']['messages'][0][0].get('content')

'स्वस्थ रहे खातिर तीन गो टिप्स दीं।'

In [ ]:
#ds = ds.map(lambda row: tokenizer(row['message']), batched=True)

In [ ]:
ds.push_to_hub("pksx01/alpaca_bhojpuri_instruction")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/52 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/pksx01/alpaca_bhojpuri_instruction/commit/dd60ccb328d45844df352f95aa3994be5754f7ef', commit_message='Upload dataset', commit_description='', oid='dd60ccb328d45844df352f95aa3994be5754f7ef', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/pksx01/alpaca_bhojpuri_instruction', endpoint='https://huggingface.co', repo_type='dataset', repo_id='pksx01/alpaca_bhojpuri_instruction'), pr_revision=None, pr_num=None)

In [4]:
import torch
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextGenerationPipeline

model_id = "sarvamai/sarvam-1"

'''bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)'''

#model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map='auto')
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/279M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/193 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/775k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [5]:
# add a chat template to the tokenizer so that it can handle multi-turn conversations
tokenizer.chat_template = """{% if messages[0]['role'] == 'system' %}
                        {% set loop_messages = messages[1:] %}
                        {% set system_message = messages[0]['content'] %}
                        {% else %}
                        {% set loop_messages = messages %}
                        {% set system_message = false %}
                        {% endif %}
                        {% for message in loop_messages %}
                        {% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}
                        {{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}
                        {% endif %}
                        {% if loop.index0 == 0 and system_message != false %}
                        {% set content = '<<SYS>>\\n' + system_message + '\\n<</SYS>>\\n\\n' + message['content'] %}
                        {% else %}
                        {% set content = message['content'] %}
                        {% endif %}
                        {% if message['role'] == 'user' %}
                        {{ bos_token + '[INST] ' + content.strip() + ' [/INST]' }}
                        {% elif message['role'] == 'assistant' %}
                        {{ ' '  + content.strip() + ' ' + eos_token }}
                        {% endif %}
                        {% endfor %}"""

tokenizer.add_tokens("[PAD]", special_tokens=True)
tokenizer.pad_token = "[PAD]"
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(68097, 2048)

Push tokenizer to HuggingFace Hub

In [6]:
tokenizer.push_to_hub("pksx01/sarvam-1-it-bhojpuri",
                  use_auth_token=True,
                  commit_message="added tokenizer",
                  private=False)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pksx01/sarvam-1-it-bhojpuri/commit/0b69ec78c8ff47056f990da79be1c20e9f3c91d4', commit_message='added tokenizer', commit_description='', oid='0b69ec78c8ff47056f990da79be1c20e9f3c91d4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/pksx01/sarvam-1-it-bhojpuri', endpoint='https://huggingface.co', repo_type='model', repo_id='pksx01/sarvam-1-it-bhojpuri'), pr_revision=None, pr_num=None)

In [ ]:
'''from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)'''

'from peft import prepare_model_for_kbit_training\n\nmodel.gradient_checkpointing_enable()\nmodel = prepare_model_for_kbit_training(model)'

In [ ]:
'''def print_trainable_parameters(model):
    all_params = 0
    trainable_params = 0
    for _, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print("Total parameters = ", all_params)
    print("Trainable parameters = ", trainable_params)
    print("% trainable parameters = ", (100 * trainable_params / all_params))'''

'def print_trainable_parameters(model):\n    all_params = 0\n    trainable_params = 0\n    for _, param in model.named_parameters():\n        all_params += param.numel()\n        if param.requires_grad:\n            trainable_params += param.numel()\n    print("Total parameters = ", all_params)\n    print("Trainable parameters = ", trainable_params)\n    print("% trainable parameters = ", (100 * trainable_params / all_params))'

In [ ]:
"""def preprocess_function(example):
    #if example is None or example["messages"] is None or example["messages"][0].get('content') is None or example["messages"][1].get('content') is None:
        #return None
    #if example.get("messages") is None or example.get("messages")[0].get("content") is None or example.get("messages")[1].get("content") is None:
        #return None
    model_ips = tokenizer.apply_chat_template(example["messages"], tokenize=False)
    tokenized_ips = tokenizer(model_ips)
    tokenized_ips["labels"] = tokenized_ips["input_ids"].copy()
    return tokenized_ips"""

def preprocess_function(example):
    if not example.get("messages"):
        return None

    # Ensure all messages have 'content' and 'role', and content is a string
    valid_messages = []
    for msg in example["messages"]:
        if isinstance(msg.get("content"), str) and msg.get("role"):
            valid_messages.append(msg)
        else:
            print(f"Skipping invalid message: {msg}")

    if not valid_messages:
        return None

    try:
        model_ips = tokenizer.apply_chat_template(valid_messages, tokenize=False)
        tokenized_ips = tokenizer(model_ips)
        tokenized_ips["labels"] = tokenized_ips["input_ids"].copy()
        return tokenized_ips
    except Exception as e:
        print(f"Error processing example: {e}")
        return None

In [ ]:
ds = ds.map(preprocess_function, remove_columns=ds["train"].column_names)

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

Skipping invalid message: {'content': None, 'role': 'assistant'}


In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 51760
    })
})

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=128,
    lora_dropout=0.0,
    target_modules=["lm_head", "k_proj", "q_proj", "v_proj" "o_proj", "gate_proj", "down_proj", "up_proj"]
    #bias="none",
    #task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 87,523,392 || all params: 2,612,615,232 || trainable%: 3.3500


In [ ]:
#from trl import SFTTrainer
import transformers

trainer = transformers.Trainer(
    model=model,
    train_dataset=ds['train'],
    #dataset_text_field="instr_op",
    args=transformers.TrainingArguments(
        num_train_epochs=1,
        save_total_limit=1,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        gradient_checkpointing=True,
        gradient_checkpointing_kwargs={"use_reentrant": False},
        warmup_steps=10,
        weight_decay=0.0001,
        max_steps=500,
        learning_rate=1e-5,
        #fp16=True,
        bf16=True,
        #save_steps=10,
        logging_steps=50,
        output_dir="sarvam-1-ft_bhojpuri"
    ),
    data_collator=transformers.DataCollatorForSeq2Seq(tokenizer)
)

#model.config.use_cache = False #enable it for inference
trainer.train()

max_steps is given, it will override any value given in num_train_epochs
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
50,3.789000
100,3.043600
150,2.742900
200,2.581400
250,2.531800
300,2.407600
350,2.320800
400,2.333700
450,2.275300
500,2.322000


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=500, training_loss=2.634796127319336, metrics={'train_runtime': 5395.7356, 'train_samples_per_second': 0.371, 'train_steps_per_second': 0.093, 'total_flos': 1.0885590955874304e+16, 'train_loss': 2.634796127319336, 'epoch': 0.03863987635239567})

In [ ]:
"""model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
model_to_save.save_pretrained("outputs")"""

'model_to_save = trainer.model.module if hasattr(trainer.model, \'module\') else trainer.model\nmodel_to_save.save_pretrained("outputs")'

In [ ]:
"""lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)"""

"lora_config = LoraConfig.from_pretrained('outputs')\nmodel = get_peft_model(model, lora_config)"

In [ ]:
"""question = input()

ips = tokenizer(question, return_tensors="pt").to("cuda:0")
ops = model.generate(**ips, max_new_tokens=50)
print(tokenizer.decode(ops[0], skip_special_tokens=True))"""

'question = input()\n\nips = tokenizer(question, return_tensors="pt").to("cuda:0")\nops = model.generate(**ips, max_new_tokens=50)\nprint(tokenizer.decode(ops[0], skip_special_tokens=True))'

In [ ]:
def generate_response(question):
    message = [{"role": "user", "content": question}]
    model_ip = tokenizer.apply_chat_template(message, tokenize=False)
    tokenized_ip = tokenizer(model_ip, return_tensors="pt").to("cuda")
    model.eval()
    op_tokens = model.generate(
        **tokenized_ip,
        max_new_tokens=250,
        temperature=0.01,
        top_k=50,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )

    op = tokenizer.decode(op_tokens[0], skip_special_tokens=True)
    return op

In [ ]:
generate_response("भारत के पहिला प्रधानमंत्री के रहे?")

'                       [INST] भारत के पहिला प्रधानमंत्री के रहे? [/INST]\n                         भारत के पहिले प्रधानमंत्री जवाहरलाल नेहरू थे। '

In [ ]:
generate_response("स्वस्थ रहे खातिर तीन गो टिप्स दीं।")

'                       [INST] स्वस्थ रहे खातिर तीन गो टिप्स दीं। [/INST]\n                         1. संतुलित आहार खाले: फल, सब्जियां, साबुत अनाज, लीन प्रोटीन और स्वस्थ वसा से भरपूर आहार खाले से शरीर के स्वास्थ्य में मदद मिलेला।\n\n2. नियमित व्यायाम करे: शारीरिक गतिविधि के कई लाभ होला, जैसे कि हृदय रोग के जोखिम कम कइल, वजन नियंत्रित कइल, तनाव कम कइल, और ऊर्जा बढ़ावे वाला।\n\n3. पर्याप्त नींद लें: नींद समग्र स्वास्थ्य के लिए महत्वपूर्ण भूमिका निभाती, खासकर जब आप स्वस्थ आहार खाले आ नियमित व्यायाम करे। नींद की कमी से वजन बढ़ सकता, तनाव बढ़ सकता, और प्रतिरक्षा प्रणाली कमजोर हो सकती।\n\n4. तनाव का प्रबंधन करे: तनाव के उच्च स्तर से स्वास्थ्य पर नकारात्मक प्रभाव पड़ सकता, खासकर अगर आप अस्वस्थ भोजन खाले, व्यायाम ना करेला, या पर्याप्त नींद ना लेवेला। तनाव को प्रबंधित करने के कई तरीके होला, जैसे कि ध्यान, योग, या प्रियजनों के साथ समय बितावेला। </s>\n'

In [ ]:
generate_response("हमनी के वायु प्रदूषण के कइसे कम कर सकेनी जा?")

'                       [INST] हमनी के वायु प्रदूषण के कइसे कम कर सकेनी जा? [/INST]\n                         वायु प्रदूषण के कमी के कई तरीका होला:\n\n1. सार्वजनिक परिवहन के इस्तेमाल से, कारपूलिंग से, साइकिल चलावे से, या पैदल चलवे से कार्बन उत्सर्जन में कमी आ सके।\n2. ऊर्जा के कुशल उपयोग करे खातिर, ऊर्जा-बचत के लाइट बल्ब इस्तेमाल करे, एसी बंद करे, एगो उपकरण के इस्तेमाल ना करे जब जरूरत ना हो, एगो उपकरण के इस्तेमाल ना करे जब जरूरत ना हो, एगो उपकरण के इस्तेमाल ना करे जब जरूरत ना हो, एगो उपकरण के इस्तेमाल ना करे जब जरूरत ना हो, एगो उपकरण के इस्तेमाल ना करे जब जरूरत ना हो, एगो उपकरण के इस्तेमाल ना करे जब जरूरत ना हो, एगो उपकरण के इस्तेमाल ना करे जब जरूरत ना हो, एगो उपकरण के इस्तेमाल ना करे जब जरूरत ना हो, एगो उपकरण के इस्तेमाल ना करे जब जरूरत ना हो, एगो उपकरण के इस्तेमाल ना करे जब जरूरत ना हो, एगो उपकरण के इस्तेमाल ना करे जब जरूरत ना हो, एगो उपकरण के इस्तेमाल ना करे जब जरूरत ना हो, एगो उपकरण के इस्तेमाल ना करे जब जरूरत ना हो, एगो उपकरण के इस्तेमाल ना करे जब जरूरत ना हो, एगो उपकरण के इस्तेमा

In [ ]:
generate_response("october k mahine me bharat me kaha kaha ghumal thik rahi?")

'                       [INST] october k mahine me bharat me kaha kaha ghumal thik rahi? [/INST]\n                         Bharat ke sabse aurat-pasand sthanon me se ek, Goa, October mausam mein apne paryatan sukhda anubhav de sakta hain. Yeh jagah suryaast ke shandaar drishyon, vividh prakriti, aur local culture ko enjoy karte hue ghoomne ke liye upyukt ha.\n\nGoa mein October mausam mein taapman auसत से adhik hota hai, lekin yeh nirantar dhoop aur umas wala hota hai. Isliye, yah sunishchit karna jaruri hai ki aapko is avadhi mein achchhi tarah se hydrated rahe.\n\nOctober mausam mein Goa mein ghoomne layak kuchh sthan:\n\n1. Colva Beach - ek prasiddha beech jahaan par retile sand aur pan'

In [ ]:
generate_response("हमार सवाल के जवाब भोजपुरी भाषा में दीं। इज़राइल देश के स्थापना कब भइल रहे और ओमे में केकर केकर महत्वपूर्ण भूमिका रहे?")

'                       [INST] हमार सवाल के जवाब भोजपुरी भाषा में दीं। इज़राइल देश के स्थापना कब भइल रहे और ओमे में केकर केकर महत्वपूर्ण भूमिका रहे? [/INST]\n                         हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार सवाल के जवाब में, हमार'

Pushing fine-tuned model to HuggingFace Hub

In [ ]:
model.push_to_hub("pksx01/sarvam-1-it-bhojpuri",
                  use_auth_token=True,
                  commit_message="basic training",
                  private=False)

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


adapter_model.safetensors:   0%|          | 0.00/908M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pksx01/sarvam-1-it-bhojpuri/commit/5ee72a50ddb174f85aec974862af04a6c1c5a30b', commit_message='basic training', commit_description='', oid='5ee72a50ddb174f85aec974862af04a6c1c5a30b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/pksx01/sarvam-1-it-bhojpuri', endpoint='https://huggingface.co', repo_type='model', repo_id='pksx01/sarvam-1-it-bhojpuri'), pr_revision=None, pr_num=None)

**Inference after loading model from HuggingFace Hub**

In [4]:
import torch
tokenizer = AutoTokenizer.from_pretrained("pksx01/sarvam-1-it-bhojpuri")
model = AutoModelForCausalLM.from_pretrained(
    "sarvamai/sarvam-1",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model.resize_token_embeddings(len(tokenizer))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(68097, 2048)

In [6]:
# Load the PEFT model
peft_model = PeftModel.from_pretrained(
    model,
    "pksx01/sarvam-1-it-bhojpuri",
    is_trainable=False
)

In [7]:
message = [{"role": "user", "content": "भारत के पहिला प्रधानमंत्री के रहे?"}]
model_ip = tokenizer.apply_chat_template(message, tokenize=False)
tokenized_ip = tokenizer(model_ip, return_tensors="pt").to("cuda")

peft_model.eval()
with torch.no_grad():
    op_tokens = peft_model.generate(
        **tokenized_ip,
        max_new_tokens=250,
        temperature=0.01,
        top_k=50,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )

op = tokenizer.decode(op_tokens[0], skip_special_tokens=True)
print(op)

                       [INST] भारत के पहिला प्रधानमंत्री के रहे? [/INST]
                         भारत के पहिले प्रधानमंत्री जवाहरलाल नेहरू थे। 
